# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
geo_0 = 'C:/Users/Gpets/Data With Python/datasets/geo_data_0.csv'
geo_1 = 'C:/Users/Gpets/Data With Python/datasets/geo_data_1.csv'
geo_2 = 'C:/Users/Gpets/Data With Python/datasets/geo_data_2.csv'

geo_0_alt = '/datasets/geo_data_0.csv'
geo_1_alt = '/datasets/geo_data_1.csv'
geo_2_alt = '/datasets/geo_data_2.csv'

def load_data(primary_path, secondary_path):
    if os.path.exists(primary_path):
        try:
            return pd.read_csv(primary_path, sep=',')
        except:
            return pd.read_csv(primary_path, sep=';')
    elif os.path.exists(secondary_path):
        try:
            return pd.read_csv(secondary_path, sep=',')
        except:
            return pd.read_csv(secondary_path, sep=';')
    else:
        print(f"Файл не найден: {primary_path} или {secondary_path}")
        return None


geo0_df = load_data(geo_0, geo_0_alt)
geo1_df = load_data(geo_1, geo_1_alt)
geo2_df = load_data(geo_2, geo_2_alt)

In [3]:
geo0_df.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
geo1_df.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
geo2_df.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Первый регион

Для начала переведем столбец `id` в индекс для обработки на дубликаты

In [6]:
geo0_df = geo0_df.set_index('id')

In [7]:
print('Кол-во строк и столбцов в geo0_df:',geo0_df.shape)
print("Число скважин:", geo0_df.index.nunique())
print('Число дубликатов',geo0_df.duplicated().sum())

Кол-во строк и столбцов в geo0_df: (100000, 4)
Число скважин: 99990
Число дубликатов 0


In [8]:
geo0_df.index[geo0_df.index.duplicated()]

Index(['HZww2', 'bxg6G', 'A5aEY', 'QcMuo', '74z30', 'AGS9W', 'Tdehs', 'fiKDv',
       'TtcGQ', 'bsk9y'],
      dtype='object', name='id')

In [9]:
geo0_df = geo0_df[~geo0_df.index.duplicated(keep='first')]

Дубликатов не наблюдается

In [10]:
geo0_df.describe()

,f0,f1,f2,product
count,99990.000000,99990.000000,99990.000000,99990.000000
mean,0.500454,0.250141,2.502629,92.499684
std,0.871844,0.504430,3.248149,44.288304
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072572,-0.200877,0.287784,56.497069
50%,0.502405,0.250252,2.515969,91.847928
75%,1.073626,0.700640,4.715035,128.563699
max,2.362331,1.343769,16.003790,185.364347


* Объем выборки: 100,000 наблюдений.


* Средние значения близки к нулю: `f0 ~ 0.5`, `f1 ~ 0.25`, `f2 ~ 2.5.`


* `f2` имеет широкий диапазон от -12.1 до 16.0, тогда как `f0` и `f1` варьируются от -1.4 до 2.4 и от -0.8 до 1.3 соответственно.


* Среднее значение запасов в скважине ~92.5 тыс. баррелей.


* Значения варьируются от 0 до 185.4 тыс. баррелей.


* Стандартное отклонение ~44.3 тыс., что указывает на умеренное разброс значений.

In [11]:
geo0_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99990 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99990 non-null  float64
 1   f1       99990 non-null  float64
 2   f2       99990 non-null  float64
 3   product  99990 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


Типы данных, соответствуют ожиданиям.

In [12]:
geo0_df.isnull().sum()

f0         0
f1         0
f2         0
product    0
dtype: int64

В данных пропусков нет

### Второй регион

Сделаем все тоже самое что и в первом регионе

In [13]:
geo1_df = geo1_df.set_index('id')

In [14]:
print('Кол-во строк и столбцов в geo0_df:',geo1_df.shape)
print("Число скважин:", geo1_df.index.nunique())
print('Число дубликатов',geo1_df.duplicated().sum())

Кол-во строк и столбцов в geo0_df: (100000, 4)
Число скважин: 99996
Число дубликатов 0


In [15]:
geo1_df.index[geo1_df.index.duplicated()]

Index(['LHZR0', 'bfPNe', 'wt4Uk', '5ltQ6'], dtype='object', name='id')

In [16]:
geo1_df = geo1_df[~geo1_df.index.duplicated(keep='first')]

In [17]:
geo1_df.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,1.141209,-4.796608,2.494501,68.823916
std,8.965815,5.119906,1.703579,45.944663
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011475,57.085625
75%,8.620964,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [18]:
geo1_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99996 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99996 non-null  float64
 1   f1       99996 non-null  float64
 2   f2       99996 non-null  float64
 3   product  99996 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [19]:
geo1_df.isnull().sum()

f0         0
f1         0
f2         0
product    0
dtype: int64

### Третий регион

In [20]:
geo2_df = geo2_df.set_index('id')

In [21]:
print('Кол-во строк и столбцов в geo0_df:',geo2_df.shape)
print("Число скважин:", geo2_df.index.nunique())
print('Число дубликатов',geo2_df.duplicated().sum())

Кол-во строк и столбцов в geo0_df: (100000, 4)
Число скважин: 99996
Число дубликатов 0


In [22]:
geo2_df.index[geo2_df.index.duplicated()]

Index(['xCHr8', 'VF7Jo', 'KUPhW', 'Vcm5J'], dtype='object', name='id')

In [23]:
geo2_df = geo2_df[~geo2_df.index.duplicated(keep='first')]

In [24]:
geo2_df.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,0.002002,-0.002159,2.495084,94.998342
std,1.732052,1.730397,3.473482,44.749573
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162328,-1.174841,0.130269,59.450028
50%,0.009424,-0.009661,2.484236,94.925026
75%,1.158477,1.163523,4.858720,130.586815
max,7.238262,7.844801,16.739402,190.029838


In [25]:
geo2_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99996 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99996 non-null  float64
 1   f1       99996 non-null  float64
 2   f2       99996 non-null  float64
 3   product  99996 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


In [26]:
geo2_df.isnull().sum()

f0         0
f1         0
f2         0
product    0
dtype: int64

## Обучение и проверка модели

Разобьем данные на тренировочную и валидационную выборки.

In [27]:
RANDOM_STATE = 228

X_0 = geo0_df[['f0', 'f1', 'f2']]
y_0 = geo0_df['product']
X_train_0, X_valid_0, y_train_0, y_valid_0 = train_test_split(
    X_0,
    y_0,
    test_size = 0.25,
    random_state = RANDOM_STATE)


X_1 = geo1_df[['f0', 'f1', 'f2']]
y_1 = geo1_df['product']
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(
    X_1,
    y_1,
    test_size = 0.25,
    random_state = RANDOM_STATE)

X_2 = geo2_df[['f0', 'f1', 'f2']]
y_2 = geo2_df['product']
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(
    X_2,
    y_2,
    test_size = 0.25,
    random_state = RANDOM_STATE)

Напишем функцию для создания пайплайна для каждого региона, включающий стандартное масштабирование признаков и линейную регрессию.

In [28]:
pipeline = Pipeline([
    ('preprocessor', StandardScaler()),
    ('models', LinearRegression())
])

Создадим пайплайн для каждого региона и обучим модели на валидацонных выборках

In [29]:
pipeline.fit(X_train_0, y_train_0)
y_pred_0 = pipeline.predict(X_valid_0)

pipeline.fit(X_train_1, y_train_1)
y_pred_1 = pipeline.predict(X_valid_1)

pipeline.fit(X_train_2, y_train_2)
y_pred_2 = pipeline.predict(X_valid_2)

Сохраним DataFrame для каждого региона, чтобы сохранить реальные и предсказанные значения.

In [30]:
predictions_0 = pd.DataFrame({'actual': y_valid_0, 'predicted': y_pred_0})
predictions_1 = pd.DataFrame({'actual': y_valid_1, 'predicted': y_pred_1})
predictions_2 = pd.DataFrame({'actual': y_valid_2, 'predicted': y_pred_2})

In [31]:
predictions_2

,actual,predicted
id,,
8kRyN,137.416387,104.818900
2NhAv,151.540415,104.982363
FmIF8,57.511246,103.575737
VAoe0,31.272420,68.712312
ZVWaa,157.034284,72.426250
...,...,...
psKSQ,18.623894,90.622315
22bSD,64.778623,63.160537
hq381,161.813034,93.619793


Выведем средний запас предсказанного сырья и RMSE модели (корень из среднеквадратичной ошибки).

In [32]:
rmse_0 = mean_squared_error(y_valid_0, y_pred_0, squared=False)
rmse_1 = mean_squared_error(y_valid_1, y_pred_1, squared=False)
rmse_2 = mean_squared_error(y_valid_2, y_pred_2, squared=False)

mean_pred_0 = np.mean(y_pred_0)
mean_pred_1 = np.mean(y_pred_1)
mean_pred_2 = np.mean(y_pred_2)

print(f"RMSE для региона 0: {rmse_0}")
print(f"Средний запас для региона 0: {mean_pred_0} \n")
print(f"RMSE для региона 1: {rmse_1}")
print(f"Средний запас для региона 1: {mean_pred_1} \n")
print(f"RMSE для региона 2: {rmse_2}")
print(f"Средний запас для региона 2: {mean_pred_2}")

RMSE для региона 0: 37.689091908751045
Средний запас для региона 0: 92.68921815442516 

RMSE для региона 1: 0.8885826896416646
Средний запас для региона 1: 68.86343359020876 

RMSE для региона 2: 39.8594402786368
Средний запас для региона 2: 94.80958298182011


### Анализ результатов по регионам

* Регион 0:
    * Среднеквадратическая ошибка: 37.69. Ошибка предсказаний относительно высокая, но средний запас в регионе (92.69 тыс. баррелей) тоже немаленький. Это говорит о том, что модель предсказывает в пределах разумного, но точность предсказаний можно улучшить.
    
    * Средний запас: 92.69 тыс. баррелей. Регион обладает значительным потенциалом запасов.
    
    Несмотря на относительно высокую ошибку, регион демонстрирует хорошие средние запасы. Возможно, стоит рассмотреть улучшение модели или дополнительные признаки для повышения точности.
    
    
* Регион 1:

    * Среднеквадратическая ошибка: 0.89. Ошибка предсказания крайне низкая, что свидетельствует о высокой точности модели.
    
    * Средний запас: 68.86 тыс. баррелей. Однако средний запас в этом регионе ниже, чем в остальных.
    
    Регион имеет самые низкие средние запасы, но высокая точность модели делает его предсказания надежными. Возможно, он менее перспективен с точки зрения добычи.
    
    
* Регион 2:

    * Среднеквадратическая ошибка: 39.86. Ошибка предсказания самая высокая из всех трёх регионов.
    
    * Средний запас: 94.81 тыс. баррелей. Средний запас немного превышает показатель региона 0, что делает его перспективным.
    
    Высокая ошибка предсказания указывает на необходимость доработки модели. Несмотря на это, регион имеет хорошие запасы, что делает его потенциально привлекательным для добычи.
    


Общий вывод:

* Регион 1 демонстрирует высокую точность предсказаний, но его средние запасы ниже, чем у остальных.

* Регионы 0 и 2 имеют значительные запасы, но требуют улучшения качества моделей для повышения точности предсказаний.

## Подготовка к расчёту прибыли

Пройдемся по условиям задачи:

* Количество лучших скважин: n = 200

* Количество исследуемых скважин : N = 500

* Бюджет на разработку: B = 10 млрд (10e9) 

* Доход с 1 тыс. баррелей: A = 450000

In [33]:
n = 200
N = 500
one_barrel = 450_000
budget = 10e9

Расчёт достаточного объёма сырья для безубыточной разработки

Стоимость разработки одной скважины мы расчитаем так: `Бюджет / Количество лучших скважин`

In [34]:
cost_per_well = budget / n
cost_per_well

50000000.0

Себестоимость одной скавижины состовляет 50 млн рублей

Теперь расчитаем достаточный объём сырья для окупаемости, для этого мы себестоимость одной скважины поделим на Доход с 1 баррели.

In [35]:
payback = cost_per_well / one_barrel
print(f"Достаточный объём сырья на одну скважину: {payback:.2f} тыс. баррелей")

Достаточный объём сырья на одну скважину: 111.11 тыс. баррелей


Сравним этот показатель со средним запасами в регионах:

* Регион 0: Средний запас (92.69) меньше порога.

* Регион 1: Средний запас (68.86) значительно ниже порога.

* Регион 2: Средний запас (94.81) меньше порога, но ближе всех к безубыточности.

Все регионы имеют средний запас сырья ниже порога безубыточности.

## Расчёт прибыли и рисков 

### Расчет прибыли

Рассчитываем прибыль на основе выбранных скважин и предсказаний модели. Будем использовать Истиные значения объёма сырья (target), предсказанные значения объёма сырья (predictions), количество лучших скважин для разработки(n_wells), доход с 1 тыс. баррелей(price_per_barrel) и общий бюджет на разработку(budget)

Напишем для нее функцию `calculate_profit`

In [36]:
def calculate_profit(target, predictions, n_wells=n, price_per_barrel=one_barrel, budget=budget):
    # Выбираем индексы 200 скважин с максимальными предсказаниями
    selected_indices = predictions.argsort()[-n_wells:]
    
    # Суммируем целевые значения объёма сырья для выбранных скважин
    selected_target_sum = target[selected_indices].sum()
    
    # Рассчитываем общий доход
    revenue = selected_target_sum * price_per_barrel
    
    # Вычисляем прибыль
    profit = revenue - budget
    
    return profit



Используем функцию и смотрим прибыль по регионам

In [37]:
# Расчёт прибыли
profit_0 = calculate_profit(predictions_0['actual'], predictions_0['predicted'])
print(f"Прибыль региона 0: {profit_0:.2f} руб.")

profit_1 = calculate_profit(predictions_1['actual'], predictions_1['predicted'])
print(f"Прибыль региона 1: {profit_1:.2f} руб.")

profit_2 = calculate_profit(predictions_2['actual'], predictions_2['predicted'])
print(f"Прибыль региона 2: {profit_2:.2f} руб.")

Прибыль региона 0: 3242805491.07 руб.
Прибыль региона 1: 2415086696.68 руб.
Прибыль региона 2: 2235668377.19 руб.


Регион 0:
* Прибыль: 3.24 млрд руб.

* Регион показывает наибольшую прибыль среди всех, что указывает на высокий потенциал разработки.


Регион 1:

* Прибыль: 2.42 млрд руб.

* Результат ниже чем у региона 0. Основное преимущество региона 1 — высокая точность модели предсказания (RMSE всего 0.89), что снижает вероятность ошибок в прогнозах.


Регион 2: 

* Прибыль: 2.24 млрд руб.

* Это наименьший показатель среди регионов. Несмотря на относительно высокие средние запасы, предсказания менее точны (RMSE около 39.86), что может снижать уверенность в прогнозах.

### Расчет рисков

In [39]:
state = np.random.RandomState(228)
profits_0 = []

for _ in range(1000):
    # Случайная выборка с возвратом
    sample = predictions_0.sample(n=500, replace=True, random_state=state)
    # Расчёт прибыли
    profit = calculate_profit(sample['actual'], sample['predicted'])
    profits_0.append(profit)
    
    
profits_0 = pd.Series(profits_0)
mean_profit_0 = profits_0.mean()
lower_0 = profits_0.quantile(0.025)
upper_0 = profits_0.quantile(0.975)
risk_0 = (profits_0 < 0).mean()

print(f"Регион 0:")
print(f"Средняя прибыль = {mean_profit_0:.2f} руб.")
print(f"95%-й доверительный интервал: [{lower_0:.2f}, {upper_0:.2f}] руб.")
print(f"Риск убытков: {risk_0:.2%}\n")




Регион 0:
Средняя прибыль = 399745773.41 руб.
95%-й доверительный интервал: [-110550905.52, 876958155.66] руб.
Риск убытков: 6.00%



In [40]:
profits_1 = []

for _ in range(1000):
    sample = predictions_1.sample(n=500, replace=True, random_state=state)
    profit = calculate_profit(sample['actual'], sample['predicted'])
    profits_1.append(profit)

profits_1 = pd.Series(profits_1)
mean_profit_1 = profits_1.mean()
lower_1 = profits_1.quantile(0.025)
upper_1 = profits_1.quantile(0.975)
risk_1 = (profits_1 < 0).mean()

print(f"Регион 1:")
print(f"Средняя прибыль = {mean_profit_1:.2f} руб.")
print(f"95%-й доверительный интервал: [{lower_1:.2f}, {upper_1:.2f}] руб.")
print(f"Риск убытков: {risk_1:.2%}\n")

Регион 1:
Средняя прибыль = 445363341.84 руб.
95%-й доверительный интервал: [53178652.47, 858713141.63] руб.
Риск убытков: 1.20%



In [42]:
profits_2 = []

for _ in range(1000):
    sample = predictions_2.sample(n=500, replace=True, random_state=state)
    profit = calculate_profit(sample['actual'], sample['predicted'])
    profits_2.append(profit)

profits_2 = pd.Series(profits_2)
mean_profit_2 = profits_2.mean()
lower_2 = profits_2.quantile(0.025)
upper_2 = profits_2.quantile(0.975)
risk_2 = (profits_2 < 0).mean()

print(f"Регион 2:")
print(f"Средняя прибыль = {mean_profit_2:.2f} руб.")
print(f"95%-й доверительный интервал: [{lower_2:.2f}, {upper_2:.2f}] руб.")
print(f"Риск убытков: {risk_2:.2%}")

Регион 2:
Средняя прибыль = 316361415.19 руб.
95%-й доверительный интервал: [-164442657.81, 812994569.44] руб.
Риск убытков: 10.30%


## Выводы: Рекомендации по выбору региона для разработки

**Регион 0:**

* Средний запас сырья: 92.69 тыс. баррелей (ниже безубыточного объёма — 111.11 тыс. баррелей).

* Средняя прибыль: 399.75 млн руб. 

* Точность предсказаний (RMSE): 37.69 (относительно высокая ошибка).

* Риск убытков:  6.00%.


Этот регион демонстрирует хорошие средние запасы сырья, однако риск убытков составляет 6%. Модель предсказаний имеет высокую ошибку, что делает прогнозы менее точными. Несмотря на это, средняя прибыль достаточно высока, что делает регион перспективным.

-------
**Регион 1:**

* Средний запас сырья: 68.86 тыс. баррелей (значительно ниже безубыточного объёма).

* Средняя прибыль: 445.36 млн руб.(наивысшая среди всех регионов).

* Точность предсказаний (RMSE): 0.89 (лучшая среди всех регионов).

* Риск убытков: 1.20%.

Этот регион имеет минимальный риск убытков и высокую точность предсказаний (RMSE всего 0.89). Однако средние запасы сырья ниже, чем в других регионах. Высокая точность модели компенсирует низкие запасы, делая прогнозы более надёжными. Регион демонстрирует наивысшую среднюю прибыль и минимальный риск, что делает его лучшим выбором.

------
**Регион 2:**

* Средний запас сырья: 94.81 тыс. баррелей (ближе к безубыточному объёму, но всё ещё ниже).

* Средняя прибыль: 316.36 млн руб. (наименьшая среди всех регионов).

* Точность предсказаний (RMSE): 39.86 (самая высокая ошибка).

* Риск убытков: 10.30%.

Средние запасы сырья в этом регионе высокие, однако прибыль наименьшая среди всех регионов. Риск убытков достигает 10.30%, а точность предсказаний оставляет желать лучшего (самая высокая RMSE). Эти факторы делают регион менее привлекательным для разработки.

------
**Рекомендации:**

* Рекомендуем выбрать регион 1 для разработки скважин.

* Причины: Регион показывает наивысшую среднюю прибыль (445.36 млн руб.) при нулевом риске убытков. Несмотря на относительно высокую ошибку предсказаний, значительные средние запасы и прибыль делают его наиболее перспективным.

* Минимальный риск убытков (1.20%)

* Высокую точность модели предсказаний (RMSE 0.89).

Несмотря на более низкие средние запасы сырья, высокая точность предсказаний и минимальный риск делают его наиболее надёжным и выгодным выбором для инвестиций.